In [1]:
%%bash
chia version
cdv --version

1.3.5
cdv, version 1.0.8


# Tic Tac Toe Coin

[tic-tac-toe puzzle](tic-tac-toe-ipynb) is an inner puzzle.

[p2_delegated_puzzle_or_hidden_puzzle.py](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/wallet/puzzles/p2_delegated_puzzle_or_hidden_puzzle.py)

In [1]:
from chia.types.blockchain_format.program import Program
from cdv.util.load_clvm import load_clvm
from clvm_tools.binutils import disassemble
from clvm_tools.clvmc import compile_clvm_text

from pathlib import Path
import sys
sys.path.insert(0, ".")
import tic_tac_toe

def load_program(file_path, search_paths):
    clsp = Path(file_path).read_text()
    return Program(
        compile_clvm_text(clsp, search_paths)
    )

tic_tac_toe_puzzle = load_program('tic-tac-toe.clsp', '.')
tic_tac_toe_coin_puzzle = load_program('tic-tac-toe-coin.clsp', '.')

In [3]:
from chia.wallet.puzzles import (singleton_top_layer, p2_delegated_puzzle_or_hidden_puzzle)

from cdv.test import Network, Wallet

network: Network = await Network.create()

await network.farm_block()

alice: Wallet = network.make_wallet("alice")
await network.farm_block(farmer=alice) # alice has 2000_000_000_000 mojos

print(f'alice: {alice.balance()}')
standard_txn_coin_wrapper = await alice.choose_coin(1_750_000_000_000)
standard_txn_coin = standard_txn_coin_wrapper.as_coin()
standard_txn_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(alice.pk())

alice: 2000000000000


In [2]:
# empty board
board = [0] * 9
# player 1 (x)
player = 1
# prepare the tic-tac-toe inner puzzle
curried_tic_tac_toe_puzzle = tic_tac_toe.get_curried_tic_tac_toe_puzzle(tic_tac_toe_puzzle, board, player)
curried_tic_tac_toe_coin_puzzle = tic_tac_toe_coin_puzzle.curry(
    tic_tac_toe_coin_puzzle,
    tic_tac_toe_puzzle, 
    curried_tic_tac_toe_puzzle)

print(f'{board} {player}')
print(curried_tic_tac_toe_puzzle.get_tree_hash())
print(curried_tic_tac_toe_coin_puzzle.get_tree_hash())
print(disassemble(curried_tic_tac_toe_coin_puzzle))
print(f'{curried_tic_tac_toe_coin_puzzle.get_tree_hash()}\n')

# player 1 plays position 4
next_player = 2
position = 4
solution = Program.to([position, next_player])
curried_tic_tac_toe_coin_puzzle = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(curried_tic_tac_toe_coin_puzzle))
print(f'{curried_tic_tac_toe_coin_puzzle.get_tree_hash()}\n')

# player 2 plays position 5
next_player = 1
position = 5
solution = Program.to([position, next_player])
curried_tic_tac_toe_coin_puzzle = curried_tic_tac_toe_coin_puzzle.run(solution)
print(disassemble(curried_tic_tac_toe_coin_puzzle))
print(f'{curried_tic_tac_toe_coin_puzzle.get_tree_hash()}\n')


[0, 0, 0, 0, 0, 0, 0, 0, 0] 1
15089968ec663f2f696effa21bc1b9d62045eb47ee4d5c9cb4a7fa7b2d901a7f
6f7a531db1e96dae108ded928f593a693e9723c42721d67fdc3ce7d80e08c661
(a (q 2 (q 2 4 (c 2 (c 5 (c (c 5 (c 11 (c (a 4 (c 2 (c 11 (c (c (f (r (a 23 (c 47 ())))) (c 95 ())) ())))) ()))) ())))) (c (q (c (q . 2) (c (c (q . 1) 5) (c (a 6 (c 2 (c 11 (q 1)))) ()))) 2 (i 5 (q 4 (q . 4) (c (c (q . 1) 9) (c (a 6 (c 2 (c 13 (c 11 ())))) ()))) (q . 11)) 1) 1)) (c (q 2 (q 2 4 (c 2 (c 5 (c (c 5 (c 11 (c (a 4 (c 2 (c 11 (c (c (f (r (a 23 (c 47 ())))) (c 95 ())) ())))) ()))) ())))) (c (q (c (q . 2) (c (c (q . 1) 5) (c (a 6 (c 2 (c 11 (q 1)))) ()))) 2 (i 5 (q 4 (q . 4) (c (c (q . 1) 9) (c (a 6 (c 2 (c 13 (c 11 ())))) ()))) (q . 11)) 1) 1)) (c (q 2 (q 2 10 (c 2 (c (a 14 (c 2 (c 5 (c 11 (c () (c 23 ())))))) (c 11 ())))) (c (q ((a (i (any (a 12 (c 2 (c 9 (c 21 (c 45 (c 11 ())))))) (a 12 (c 2 (c 93 (c -67 (c 381 (c 11 ())))))) (a 12 (c 2 (c 765 (c 1533 (c 3069 (c 11 ())))))) (a 12 (c 2 (c 9 (c 93 (c 765 (c 11 ())))))) 